# MILP Formulation of "Syncup Sessions"

> Note: For simplicity of implementation, this document will consistently use zero-based numbering. All ranges (m, n) are inclusive of m and exclusive of n.


In [ ]:
# Install dependencies
!pip3 install scipy
!pip3 install numpy
!pip3 install sympy
!pip3 install matplotlib
!pip3 install networkx

In [ ]:
# Imports
from typing import List, Tuple, Iterable, Union, NamedTuple, Literal
from random import Random
import sys

from sympy import Matrix, init_printing
import numpy as np
import scipy as sp
import networkx as nx
from IPython.display import Latex


from IPython.display import Math


### 1. MILP Formulation and Implementation


Let *s* be the number of students, *t* be the number of TAs, *p* be the number of professors, and let the latter two classes be collectively known as instructors *i* = *t* + *p*. Let *d* be a matrix s.t. *d<sub>j,k</sub>* is the time required for instructor *j* and student *k* to hold a session.

Let |*x*| = *si* + 1 where:
* *x<sub>s * j + k</sub>* is a binary variable denoting whether a session is held between instructor *j* and student *k*.
* *x<sub>s * i</sub>* is an unbounded real variable.

Let |*c*| also = *si* + 1 where *c* is a series of `0`s with a `1` in the last position. This will minimize only the very last variable.

The following value constraints shall be applied:
* All values in *x* save for *x<sub>si</sub>* are ∈ (0, 1).
    * To satisfy the binary constraint.
* The sum of all variables *x<sub>s * j + k</sub>* for a student *k* for all *j* must be 1.
    * Each student must get exactly one session held.
* *x<sub>si</sub>* must be greater than or equal to the sum of durations of each instructor.
    * In other words, the sum of all durations of each instructor minus *x<sub>si</sub>* must be less than or equal to 0.

The objective goal is to minimize the maximum amount of time spent by all instructors.

**Expresed mathematically**:

(Please use your latex renderer for this if you're on Google Colab. Colab, amazingly, does not support Latex.)

In [ ]:
%%latex
\begin{align}

\text{Minimize }    & x_{si} \nonumber \\
\text{subject to }  & \forall_{{l}\in \lbrack 0, si\rparen}   & x_{l} \in \{0, 1\} \nonumber \\
                    & \forall_{{j}\in{i}}   & \sum^{s}_{k=0}(d_{s * j + k}x_{s * j + k}) - x_{si} \le 0 \nonumber \\ 
                    & \forall_{{k}\in{s}}   & \sum^{i}_{j=0}(x_{s * j + k}) = 1 \nonumber
\end{align}



> References:
>
> \[1\] S. Datta, “Using Min/Max Within an Integer Linear Program,” *Baeldung CS*, Jan. 18, 2021. https://www.baeldung.com/cs/ilp-max-min (accessed Dec. 22, 2022).

#### Code
The following code implements assisting data structures, brute force and the MILP.

In [ ]:
# This code converts the input set of variables to a format accepted by
# the HiGHS optimizer, accessible via SciPy linprog. (https://highs.dev/) 

# Unlike the simplex form, there are two sets of constraints:
## eq: Equality constraints
## ub: Upper-bound constraints (≤)
class LinearProgram(NamedTuple):
    A_eq: np.ndarray
    b_eq: np.ndarray
    A_ub: np.ndarray
    b_ub: np.ndarray
    c: np.ndarray
    constrain_int: Iterable[int]
    bounds: Tuple[float, float]

    def as_kwargs(self) -> dict:
        retval = self._asdict().copy()
        retval["integrality"] = self.constrain_int
        del retval["constrain_int"]
        return retval

    def to_latex(self) -> str:
        latex_str = "\\begin{align}"

        n = len(self.c)

        minimize_str = "\\text{Minimize } & "
        first_added = False
        for l in range(0, n):
            if self.c[l] != 0:
                if not first_added:
                    first_added = True
                elif self.c[l] > 0:
                    minimize_str += "+"
                minimize_str += f"{self.c[l]}x_{{{l}}}"
        latex_str += f"\n & {minimize_str} \\\\"

        latex_str += "\n\\text{subject to} & & \\\\"

        for coefficients, rhs in zip(self.A_eq, self.b_eq):
            coeff_str = ""
            first_added = False
            for i, coeff in enumerate(coefficients):
                if coeff != 0:
                    if not first_added:
                        first_added = True
                    elif coeff > 0:
                        latex_str += "+"
                    latex_str += f"{coeff}x_{{{i}}}"
            coeff_str += " & = "
            coeff_str += f"{rhs}"
            latex_str += f"\n & {coeff_str}\\\\"

        for coefficients, rhs in zip(self.A_ub, self.b_ub):
            coeff_str = ""
            first_added = False
            for i, coeff in enumerate(coefficients):
                if coeff != 0:
                    if not first_added:
                        first_added = True
                    elif coeff > 0:
                        latex_str += "+"
                    latex_str += f"{coeff}x_{{{i}}}"
            coeff_str += " & \\le "
            coeff_str += f"{rhs}"
            latex_str += f"\n & {coeff_str}\\\\"


        latex_str += "\n\\end{align}"
        return latex_str

class Favorites(NamedTuple):
    fav_prof: int
    fav_prof_duration: float
    fav_ta: int
    fav_ta_duration: float


class Session(NamedTuple):
    student: int
    instructor_class: Union[Literal["Professor"], Literal["TA"]]
    instructor: int
    duration: float

    def __repr__(self) -> str:
        return f"<Session student {self.student} - {self.instructor_class.lower()} {self.instructor} ({self.duration})>"

    @classmethod
    def to_graph(Self, sessions: List["Session"]):
        G = nx.Graph()
        for session in sessions:
            student = f"S{session.student}"
            instructor = f"{session.instructor}"
            if session.instructor_class == "TA":
                instructor = f"T{instructor}"
            else:
                instructor = f"P{instructor}"

            G.add_edge(student, instructor, weight=session.duration)
        return G

    @classmethod
    def draw(Self, sessions: List["Session"]):
        G = Self.to_graph(sessions)
        pos = nx.spring_layout(G, k=2)
        nx.draw_networkx(G, pos)
        nx.draw_networkx_edge_labels(
            G, pos, edge_labels=nx.get_edge_attributes(G, "weight")
        )


class Classroom(object):
    def __init__(self, p: int, t: int, sp: List[Favorites]) -> None:
        """
        p:          The number of professors.
        t:          The number of TAs.
        sp:         A list of students' preferences; their favorite professor \
                    and the time taken to conduct a session with them.
        """
        self.p = p
        self.t = t

        self.i = p + t
        self.s = len(sp)
        self.sp = sp
        
        d = np.zeros((self.s, self.i))
        for k, preference in enumerate(self.sp):
            d[k][preference.fav_prof] = preference.fav_prof_duration
            d[k][p + preference.fav_ta] = preference.fav_ta_duration
        d.flags.writeable = False
        self.d = d


    def to_lp(self) -> LinearProgram:
        si = self.s * self.i
        n = si + 1  # Variable count

        m_eq = self.s   # Equality Constraint Count: One per student
        m_ub = self.i   # Inequality Constraint Count: One per instructor

        A_eq = np.zeros((m_eq, n))
        b_eq = np.zeros(m_eq)

        A_ub = np.zeros((m_ub, n))
        b_ub = np.zeros(m_ub)

        c = np.zeros(n)
        c[si] = 1

        constrain_int = [1] * n
        constrain_int[si] = 0

        bounds = [(0, 1)] * n
        bounds[si] = (None, None)

        students_by_instructor: List[List[int]] = []
        for _ in range(0, self.i):
            students_by_instructor.append([])

        # The Session Guarantee Constraints
        # Each student gets exactly one session.
        for k, preference in enumerate(self.sp):
            ta_true_index = self.p + preference.fav_ta

            prof_start_index = preference.fav_prof * self.s
            ta_start_index = ta_true_index * self.s

            A_eq[k][prof_start_index + k] = 1
            A_eq[k][ta_start_index + k] = 1

            b_eq[k] = 1
            
            students_by_instructor[preference.fav_prof].append(k)
            students_by_instructor[ta_true_index].append(k)


        # The Inequality Constraints
        for j in range(self.i):
            start_idx = j * self.s

            fans = students_by_instructor[j]
            A_ub[j][si] = -1
            for k in fans:
                A_ub[j][start_idx + k] = self.d[k][j]
            
            b_ub[j] = 0 # Redundant but better be explicit

        return LinearProgram(
            A_eq=A_eq,
            b_eq=b_eq,
            A_ub=A_ub,
            b_ub=b_ub,
            c=c,
            constrain_int=constrain_int,
            bounds=bounds,
        )

    def solve_lp(self) -> Tuple[List[Session], int]:
        lp = self.to_lp()
        res = sp.optimize.linprog(**lp.as_kwargs())
        if res.x is None:
            raise ValueError(res.message)

        results = []
        x = res.x

        for j in range(0, self.p):
            start = j * self.s
            slice = x[start : start + self.s]
            for k, session in enumerate(slice):
                if session != 0:
                    duration = self.d[k][j]
                    results.append(Session(k, "Professor", j, duration))
        for j in range(0, self.t):
            start = (self.p + j) * self.s
            slice = x[start : start + self.s]
            for k, session in enumerate(slice):
                if session != 0:
                    duration = self.d[k][self.p + j]
                    results.append(Session(k, "TA", j, duration))

        return (results, res.fun)

    def solve_brute_force(self) -> Tuple[List[Session], int]:
        current_string = 2**self.s - 1

        best_assignment = []
        min_max_load = np.inf

        while current_string >= 0:
            mutable = current_string
            loads = np.zeros(self.i)
            sessions = []
            for k, preference in enumerate(self.sp):
                prof_idx = preference.fav_prof
                ta_idx = self.p + preference.fav_ta
                prof = (mutable & 1) == 1
                mutable >>= 1
                if prof:
                    sessions.append(
                        Session(
                            k,
                            "Professor",
                            preference.fav_prof,
                            preference.fav_prof_duration,
                        )
                    )
                    loads[prof_idx] += preference.fav_prof_duration
                else:
                    sessions.append(
                        Session(k, "TA", preference.fav_ta, preference.fav_ta_duration)
                    )
                    loads[ta_idx] += preference.fav_ta_duration

            max_load = max(loads)
            if max_load <= min_max_load:
                min_max_load = max_load
                best_assignment = sessions

            current_string -= 1

        return (best_assignment, min_max_load)

In [ ]:
# Trivial example: 2 professors, 2 tas, four students, all the same duration
simple_classroom = Classroom(
    p=2,
    t=2,
    sp=[
        Favorites(0, 1, 0, 1),
        Favorites(0, 1, 1, 1),
        Favorites(1, 1, 0, 1),
        Favorites(1, 1, 1, 1),
    ],
)

In [ ]:
print("-- Brute Force --")
sessions, maximum = simple_classroom.solve_brute_force()
print(maximum)
Session.draw(sessions)

In [ ]:
print("-- Linear Program --")
try:
    sessions, maximum = simple_classroom.solve_lp()
    print(maximum)
    Session.draw(sessions)
except ValueError as e:
    print(e, file=sys.stderr)

## 2. The Classroom Generator


Using a seeded RNG (for replicability):
* A number of professors from 1-3 is generated
* A number of TAs from 1-8 is generated.
* A number of students max(professor count, TA count)-parameter is generated.

Each student then randomly gets both a favorite professor and a favorite TA, and a duration from 1 to 11 time units is assigned.


This also includes a brute-force algorithm, I'm only going to use up to 10 students, because the brute-force algorithm is O(2<sup>s</sup>).

In [ ]:
def generate_classroom(seed: int = 0xFFFFFFFF, max_students: int = 11) -> Classroom:
    rng = Random(seed)

    p = rng.randrange(1, 4)
    t = rng.randrange(1, 8)
    s = rng.randrange(max(p, t), max_students)

    favorites = []

    for _ in range(0, s):
        fav_prof = rng.randrange(0, p)
        fav_prof_duration = 10 * rng.random() + 1
        fav_ta = rng.randrange(0, t)
        fav_ta_duration = 10 * rng.random() + 1
        favorites.append(Favorites(fav_prof, fav_prof_duration, fav_ta, fav_ta_duration))

    return Classroom(p, t, favorites)

        
for seed in range(0xF0, 0x10A):
    print(f"Verifying classroom generated with seed 0x{seed:x}...")
    crand = generate_classroom(seed)
    print(f"Generated a classrom with {crand.p} professors, {crand.t} TAs and {crand.s} students.")
    _, lp_max = crand.solve_lp()
    _, bf_max = crand.solve_brute_force()
    assert abs(lp_max - bf_max) / lp_max <= 0.000001 # Account for floating point errors
    print("Verified!")




## Approximation
This is a super-simple greedy algorithm: every student's preference is iterated upon and by simply checking which of the two favorites would get less workload when the current student is added to their workloads, whichever one gets a lower workload is fixed.

The simple classroom above returns an unusually high factor of 2 (as professor 2 may not be reassigned after the fact), but, nevertheless, on sufficiently large/randomized data-sets the factor is well below that, ≈1.2, as the data below shows.

The approximation runs in O(s) time and takes O(s+i) memory.

In [ ]:
def approximation_algorithm_iter1(classroom: Classroom) -> Tuple[List[Session], int]:
    sessions = []
    loads = np.zeros(classroom.i)
    for k, preference in enumerate(classroom.sp):
        prof_idx = preference.fav_prof
        ta_idx = classroom.p + preference.fav_ta

        prospective_prof_load = loads[prof_idx] + preference.fav_prof_duration
        prospective_ta_load = loads[ta_idx] + preference.fav_ta_duration
        if prospective_prof_load <= prospective_ta_load:
            sessions.append(
                Session(
                    k, "Professor", preference.fav_prof, preference.fav_prof_duration
                )
            )
            loads[prof_idx] = prospective_prof_load
        else:
            sessions.append(
                Session(k, "TA", preference.fav_ta, preference.fav_ta_duration)
            )
            loads[ta_idx] = prospective_ta_load
    return (sessions, max(loads))

print("-- Approximation Iteration 1 --")
_, simple_classroom_factor = approximation_algorithm_iter1(simple_classroom)
print(f"Factor for a simple classroom: {simple_classroom_factor}")
max_factor = simple_classroom_factor
total_factors = max_factor
factor_count = 1
for seed in range(0x1FF, 0x1FF + 500):
    crand = generate_classroom(seed, max_students=50)
    print(f"Generated a classroom with seed 0x{seed:x}: {crand.p} professors, {crand.t} TAs and {crand.s} students.")
    print("Running approximation...")
    _, approx_max = approximation_algorithm_iter1(crand)
    print("Running linear program...")
    _, lp_max = crand.solve_lp()
    factor = approx_max / lp_max
    print(f"Done: Factor {factor}")
    total_factors += factor
    factor_count += 1
    max_factor = max(max_factor, factor)
print("-" * 20)
print(f"Comparison done.")
print(f"Max factor: {max_factor}")
print(f"Average factor: {total_factors / factor_count}")

Done: Factor 1.2220373714997452
Generated a classroom with seed 0x264: 1 professors, 2 TAs and 37 students.
Running approximation...
Running linear program...
Done: Factor 1.2791223585488538
Generated a classroom with seed 0x265: 2 professors, 2 TAs and 9 students.
Running approximation...
Running linear program...
Done: Factor 1.5079354894268382
Generated a classroom with seed 0x266: 2 professors, 2 TAs and 40 students.
Running approximation...
Running linear program...
Done: Factor 1.0984436304460332
Generated a classroom with seed 0x267: 1 professors, 4 TAs and 8 students.
Running approximation...
Running linear program...
Done: Factor 1.0000000000000007
Generated a classroom with seed 0x268: 1 professors, 6 TAs and 34 students.
Running approximation...
Running linear program...
Done: Factor 1.3782984608612294
Generated a classroom with seed 0x269: 3 professors, 4 TAs and 33 students.
Running approximation...
Running linear program...
Done: Factor 1.1294133659032828
Generated a clas